In [280]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import nbformat
import plotly.express as px
from dash import Dash, dcc, html

In [293]:
df = pd.read_excel("US CPI/CPI 1913-2024.xlsx", engine='openpyxl')

In [282]:
df.index = df['Year']
df.drop(columns=['HALF1','HALF2','Year'], axis=1, inplace=True)

In [283]:
values_flat = pd.DataFrame(df.values.flatten())
MoMCPI = values_flat.pct_change(periods=1) *100
months = df.columns
years = df.index
new_index = [f"{year}-{month}" for year in years for month in months]
MoMCPI.index = new_index


YoYCPI = values_flat.pct_change(periods=12)*100
YoYCPI.index = new_index

C:\Users\jleon\AppData\Local\Temp\ipykernel_32172\3889986441.py:2: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\jleon\AppData\Local\Temp\ipykernel_32172\3889986441.py:9: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [284]:
MoMCPIFig = px.line(MoMCPI, x=MoMCPI.index, y=MoMCPI.columns, title='MoM CPI in %')
MoMCPIFig.show()

In [285]:
YoYCPIFig = px.line(YoYCPI, x=YoYCPI.index, y=YoYCPI.columns, title='YoY CPI in %')
YoYCPIFig.show()

In [286]:
ThreeMoMRollingAvg = MoMCPI.rolling(window= 3).mean()
SixMoMRollingAvg = MoMCPI.rolling(window= 6).mean()
NineMoMRollingAvg = MoMCPI.rolling(window= 9).mean()
OneYearMoMRollingAvg = MoMCPI.rolling(window= 12).mean()
CombinedMoM = pd.concat([ThreeMoMRollingAvg, SixMoMRollingAvg, NineMoMRollingAvg, OneYearMoMRollingAvg], axis=1)
CombinedMoM.columns = ['3 Month Avg', '6 Month Avg', '9 Month Avg', '1 Year Avg']
MoMSignals = px.line(CombinedMoM, x=CombinedMoM.index, y=CombinedMoM.columns, title='CPI Rolling Averages')


for i in range(CombinedMoM.shape[0]):
    check1 = [CombinedMoM[f'{x}'].iloc[i] for x in CombinedMoM.columns[1:4]]
    # check2 = [CombinedMoM[f'{x}'].iloc[i] for x in CombinedMoM.columns[2:4]]
    if np.all([CombinedMoM[f'{x} Month Avg'].iloc[i] >= value for x in [3, 6] for value in check1]) and CombinedMoM['9 Month Avg'].iloc[i] <= CombinedMoM['1 Year Avg'].iloc[i]:
        MoMSignals.add_vline(x=CombinedMoM.index[i], line_width=1.5,line_color='red')
    if np.all([CombinedMoM[f'{x} Month Avg'].iloc[i] <= value for x in [3, 6] for value in check1]) and CombinedMoM['9 Month Avg'].iloc[i] >= CombinedMoM['1 Year Avg'].iloc[i]:
        MoMSignals.add_vline(x=CombinedMoM.index[i], line_width=1.5,line_color='blue')
MoMSignals.add_hrect(y0=0.166666,y1=0.2083333, line_width=0, fillcolor='red',opacity=0.2)
MoMSignals.show()

In [ ]:
ThreeYoYRollingAvg = YoYCPI.rolling(window= 3).mean()
SixYoYRollingAvg = YoYCPI.rolling(window= 6).mean()
NineYoYRollingAvg = YoYCPI.rolling(window= 9).mean()
OneYearYoYRollingAvg = YoYCPI.rolling(window= 12).mean()
CombinedYoY = pd.concat([ThreeYoYRollingAvg, SixYoYRollingAvg, NineYoYRollingAvg, OneYearYoYRollingAvg], axis=1)
CombinedYoY.columns = ['3 Month Avg', '6 Month Avg', '9 Month Avg', '1 Year Avg']
YoYSignals = px.line(CombinedYoY, x=CombinedYoY.index, y=CombinedYoY.columns)
YoYSignals.update_layout(title= 'YoY CPI Signals',
                         xaxis_title='Date',
                         yaxis_title='CPI Average (%)',
                         legend_title='Averages',
                         template='simple_white')
YoYSignals.add_hrect(y0=2,y1=2.5, line_width=0, fillcolor='red',opacity=0.2)

for i in range(CombinedYoY.shape[0]):
    check1 = [CombinedYoY['6 Month Avg'].iloc[i],CombinedYoY['9 Month Avg'].iloc[i],CombinedYoY['1 Year Avg'].iloc[i]]
    check2 = [CombinedYoY['9 Month Avg'].iloc[i],CombinedYoY['1 Year Avg'].iloc[i]]
    if all( CombinedYoY['3 Month Avg'].iloc[i] > value for value in check1) and all( CombinedYoY['6 Month Avg'].iloc[i] > value for value in check2) and CombinedYoY['9 Month Avg'].iloc[i] <= CombinedYoY['1 Year Avg'].iloc[i]:
        YoYSignals.add_vline(x=CombinedYoY.index[i], line_width=0.5,line_color='red')
    if all( CombinedYoY['3 Month Avg'].iloc[i] < value for value in check1) and all( CombinedYoY['6 Month Avg'].iloc[i] < value for value in check2) and CombinedYoY['9 Month Avg'].iloc[i] >= CombinedYoY['1 Year Avg'].iloc[i]:
        YoYSignals.add_vline(x=CombinedYoY.index[i], line_width=0.5,line_color='blue')

YoYSignals.show()
YoYSignals.write_html('YoYSignals.html')

TypeError: line() got an unexpected keyword argument 'xaxis_title'

In [295]:
# Dash App for the visualization of the data
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nbformat
import plotly.express as px
from dash import Dash, dcc, html, Input, Output


# Create the Dash app
app = Dash(__name__)

# Load the data
script_dir = os.getcwd()

# Construct the file path
file_path = os.path.join(script_dir, "US CPI\\CPI 1913-2024.xlsx")

# Load the data
df = pd.read_excel(file_path, engine='openpyxl')

# Format the data
df.index = df['Year']

df.drop(columns=['HALF1','HALF2','Year'], 
        axis=1, 
        inplace=True)

# Calculate the MoM and YoY CPI
values_flat = pd.DataFrame(df.values.flatten())
MoMCPI = values_flat.pct_change(periods=1) *100
YoYCPI = values_flat.pct_change(periods=12)*100
months = df.columns
years = df.index
new_index = [f"{year}-{month}" for year in years for month in months]
MoMCPI.index = new_index
YoYCPI.index = new_index

#Start the Dash app
app.layout = html.Div([

    html.H1("CPI Signals",style={'text-align': 'center'}),

    dcc.Dropdown(id='dropdown',
                 options=[{'label': 'MoM Signals', 'value': 'MoM'},
                          {'label': 'YoY Signals', 'value': 'YoY'}],
                    style = {'width: 50%'},
                    value='MoM'
                    ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='CPI_Signals', figure={})
    ])

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='CPI_Signals', component_property='figure')],
    [Input(component_id='dropdown', component_property='value')]
)
def update(option_selected):
    container = "The number of Hot Signals for MoM is {} and Cold Signals is {}.".format(CPIMoMHot, CPIMoMCold)
    print(option_selected)
    print(type(option_selected))
    
    # Plotly
    if option_selected == 'MoM':
        ## Month Over Month Rolling Averagse
        CombinedMoM = pd.DataFrame()
        for x in [3, 6, 9, 12]:
            CombinedMoM[f'{x} Month Avg'] = MoMCPI.rolling(window=x).mean()
            
        ## MoM Graph
        Signal = px.line(CombinedMoM, x=CombinedMoM.index, y=CombinedMoM.columns, title='MoM CPI Rolling Averages')
        Signal.add_hrect(y0=0.166666,
                             y1=0.2083333,
                             line_width=0,
                             fillcolor='red',
                             opacity=0.2) # Monthly Target Inflation Rate based on Fed 2% Annual Target
        
        Signal.update_layout(title= 'MoM CPI Signals',
                         xaxis_title='Date',
                         yaxis_title='CPI Average (%)',
                         legend_title='Averages',
                         template='simple_white')
        ## Signal lines
        CPIMoMHot = 0
        CPIMoMCold = 0
        for i in range(CombinedMoM.shape[0]):
            check1 = [CombinedMoM[f'{x}'].iloc[i] for x in CombinedMoM.columns[1:4]]
            if np.all([CombinedMoM[f'{x} Month Avg'].iloc[i] >= value for x in [3, 6] for value in check1]) and CombinedMoM['9 Month Avg'].iloc[i] <= CombinedMoM['1 Year Avg'].iloc[i]:
                Signal.add_vline(x=CombinedMoM.index[i], line_width=1.5,line_color='red')
                CPIMoMHot += 1
            if np.all([CombinedMoM[f'{x} Month Avg'].iloc[i] <= value for x in [3, 6] for value in check1]) and CombinedMoM['9 Month Avg'].iloc[i] >= CombinedMoM['1 Year Avg'].iloc[i]:
                Signal.add_vline(x=CombinedMoM.index[i], line_width=1.5,line_color='blue') 
                CPIMoMCold += 1

    if option_selected == 'YoY':
         ## Year Over Year Rolling Averages
        CombinedYoY = pd.DataFrame()
        for x in [3, 6, 9, 12]:
            CombinedYoY[f'{x} Month Avg'] = YoYCPI.rolling(window=x).mean()
        
        ## YoY Graph
        Signal = px.line(CombinedYoY, x=CombinedYoY.index, y=CombinedYoY.columns, title='YoY CPI Rolling Averages')
        Signal.add_hrect(y0=1.5,
                             y1=2.5,line_width=0,
                             fillcolor='red',
                             opacity=0.2)
        
        Signal.add_hrect(y0=0.166666,
                             y1=0.2083333,
                             line_width=0,
                             fillcolor='red',
                             opacity=0.2) # 2% (+- 0.5%) Fed Long-term Inflation Target
        
        ## Signal lines
        CPIYoYHot = 0
        CPIYoYCold = 0
        for i in range(CombinedYoY.shape[0]):
            check1 = [CombinedYoY[f'{x}'].iloc[i] for x in CombinedYoY.columns[1:4]]
            if np.all([CombinedYoY[f'{x} Month Avg'].iloc[i] >= value for x in [3, 6] for value in check1]) and CombinedYoY['9 Month Avg'].iloc[i] <= CombinedYoY['1 Year Avg'].iloc[i]:
                Signal.add_vline(x=CombinedYoY.index[i], line_width=1.5,line_color='red')
                CPIYoYHot += 1
            if np.all([CombinedYoY[f'{x} Month Avg'].iloc[i] <= value for x in [3, 6] for value in check1]) and CombinedYoY['9 Month Avg'].iloc[i] >= CombinedYoY['1 Year Avg'].iloc[i]:
                Signal.add_vline(x=CombinedYoY.index[i], line_width=1.5,line_color='blue') 
                CPIYoYCold += 1
    return container, Signal

#---------
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\jleon\AppData\Local\Temp\ipykernel_32172\1694273175.py:32: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\jleon\AppData\Local\Temp\ipykernel_32172\1694273175.py:33: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: Object of type set is not JSON serializable

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: Object of type set is not JSON serializable

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: Object of type set is not JSON serializable

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: Object of type set is not JSON serializable

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: Objec